In [16]:
import wallaroo
from wallaroo.object import EntityNotFoundError

import pyarrow as pa

from wallaroo.framework import Framework

# used to display dataframe information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)
import time

In [2]:
wallaroo.__version__

'2023.3.0+f2e5950e8'

In [3]:
# Login through local Wallaroo instance

wl = wallaroo.Client()

Just for the sake of this tutorial, we'll use the SDK below to create our workspace , assign as our **current workspace**, then display all of the workspaces we have at the moment.  We'll also set up for our models and pipelines down the road, so we have one spot to change names to whatever fits your organization's standards best.

To allow this tutorial to be run multiple times or by multiple users in the same Wallaroo instance, a random 4 character prefix will be added to the workspace, pipeline, and model.

When we create our new workspace, we'll save it in the Python variable `workspace` so we can refer to it as needed.

In [4]:
import string
import random

# make a random 4 character prefix
suffix= ''.join(random.choice(string.ascii_lowercase) for i in range(4))


suffix='john'


workspace_name = f'arm-classification-finserv{suffix}'
pipeline_name = 'arm-classification-example'
model_name = 'ccfraudmodel'
model_file_name = './ccfraud.onnx'

In [5]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(name)
    return pipeline

In [6]:
workspace = get_workspace(workspace_name)

wl.set_current_workspace(workspace)

{'name': 'arm-classification-finservjohn', 'id': 32, 'archived': False, 'created_by': '26a792cb-25d0-470a-aba2-9b4c4ba373ac', 'created_at': '2023-08-15T21:27:27.693789+00:00', 'models': [], 'pipelines': []}

## Upload a model

Our workspace is created.  Let's upload our credit card fraud model to it.  This is the file name `ccfraud.onnx`, and we'll upload it as `ccfraudmodel`.  The credit card fraud model is trained to detect credit card fraud based on a 0 to 1 model:  The closer to 0 the less likely the transactions indicate fraud, while the closer to 1 the more likely the transactions indicate fraud.


Since we're already in our default workspace `ccfraudworkspace`, it'll be uploaded right to there.  Once that's done uploading, we'll list out all of the models currently deployed so we can see it included.

In [7]:
ccfraud_model = wl.upload_model(model_name, model_file_name, wallaroo.framework.Framework.ONNX).configure()

We can verify that our model was uploaded by listing the models uploaded to our Wallaroo instance with the `list_models()` command.  Note that since we uploaded this model before, we now have different versions of it we can use for our testing.

## Create a Pipeline

With our model uploaded, time to create our pipeline and deploy it so it can accept data and process it through our `ccfraudmodel`.  We'll call our pipeline `ccfraudpipeline`.

* **NOTE**:  Pipeline names must be unique.  If two pipelines are assigned the same name, the new pipeline is created as a new **version** of the pipeline.

In [12]:
pipeline = get_pipeline(pipeline_name)
pipeline.clear()

name,arm-classification-example
created,2023-08-15 21:27:31.128328+00:00
last_updated,2023-08-15 21:27:31.128328+00:00
deployed,(none)
tags,
versions,a8d55b45-2331-4d44-aeba-051eaf20e98c
steps,


Now our pipeline is set.  Let's add a single **step** to it - in this case, our `ccfraud_model` that we uploaded to our workspace.

In [13]:
pipeline.add_model_step(ccfraud_model)

name,arm-classification-example
created,2023-08-15 21:27:31.128328+00:00
last_updated,2023-08-15 21:27:31.128328+00:00
deployed,(none)
tags,
versions,a8d55b45-2331-4d44-aeba-051eaf20e98c
steps,


In [10]:
from wallaroo.engine_config import Architecture
deployment_config = wallaroo.deployment_config.DeploymentConfigBuilder().cpus(1).memory('1Gi').build()
deployment_config

{'engine': {'cpu': 1,
  'resources': {'limits': {'cpu': 1, 'memory': '1Gi'},
   'requests': {'cpu': 1, 'memory': '1Gi'}}},
 'enginelb': {},
 'engineAux': {'images': {}},
 'node_selector': {}}

In [14]:
pipeline.deploy(deployment_config=deployment_config)

Waiting for deployment - this will take up to 45s ......... ok


name,arm-classification-example
created,2023-08-15 21:27:31.128328+00:00
last_updated,2023-08-15 21:27:54.490687+00:00
deployed,True
tags,
versions,"3f2b6bf2-14eb-4005-80a9-6b0f06ded120, a8d55b45-2331-4d44-aeba-051eaf20e98c"
steps,ccfraudmodel


In [19]:
start_time = time.time()
result = pipeline.infer_from_file('./cc_data_10k.arrow')
end_time = time.time()
x64_time = end_time - start_time

outputs =  result.to_pandas()
display(outputs)

,time,in.tensor,out.dense_1,check_failures
0,2023-08-15 21:33:52.851,"[-1.0603298, 2.3544967, -3.5638788, 5.138735, -1.2308457, -0.76878244, -3.5881228, 1.8880838, -3.2789674, -3.9563255, 4.099344, -5.653918, -0.8775733, -9.131571, -0.6093538, -3.7480276, -5.0309124, -0.8748149, 1.9870535, 0.7005486, 0.9204423, -0.10414918, 0.32295644, -0.74181414, 0.038412016, 1.0993439, 1.2603409, -0.14662448, -1.4463212]",[0.99300325],0
1,2023-08-15 21:33:52.851,"[-1.0603298, 2.3544967, -3.5638788, 5.138735, -1.2308457, -0.76878244, -3.5881228, 1.8880838, -3.2789674, -3.9563255, 4.099344, -5.653918, -0.8775733, -9.131571, -0.6093538, -3.7480276, -5.0309124, -0.8748149, 1.9870535, 0.7005486, 0.9204423, -0.10414918, 0.32295644, -0.74181414, 0.038412016, 1.0993439, 1.2603409, -0.14662448, -1.4463212]",[0.99300325],0
2,2023-08-15 21:33:52.851,"[-1.0603298, 2.3544967, -3.5638788, 5.138735, -1.2308457, -0.76878244, -3.5881228, 1.8880838, -3.2789674, -3.9563255, 4.099344, -5.653918, -0.8775733, -9.131571, -0.6093538, -3.7480276, -5.0309124, -0.8748149, 1.9870535, 0.7005486, 0.9204423, -0.10414918, 0.32295644, -0.74181414, 0.038412016, 1.0993439, 1.2603409, -0.14662448, -1.4463212]",[0.99300325],0
3,2023-08-15 21:33:52.851,"[-1.0603298, 2.3544967, -3.5638788, 5.138735, -1.2308457, -0.76878244, -3.5881228, 1.8880838, -3.2789674, -3.9563255, 4.099344, -5.653918, -0.8775733, -9.131571, -0.6093538, -3.7480276, -5.0309124, -0.8748149, 1.9870535, 0.7005486, 0.9204423, -0.10414918, 0.32295644, -0.74181414, 0.038412016, 1.0993439, 1.2603409, -0.14662448, -1.4463212]",[0.99300325],0
4,2023-08-15 21:33:52.851,"[0.5817662, 0.09788155, 0.15468194, 0.4754102, -0.19788623, -0.45043448, 0.016654044, -0.025607055, 0.09205616, -0.27839172, 0.059329946, -0.019658541, -0.42250833, -0.12175389, 1.5473095, 0.23916228, 0.3553975, -0.76851654, -0.7000849, -0.11900433, -0.3450517, -1.1065114, 0.25234112, 0.020944182, 0.21992674, 0.25406894, -0.04502251, 0.10867739, 0.25471792]",[0.0010916889],0
...,...,...,...,...
10249,2023-08-15 21:33:52.851,"[-0.90164274, -0.50116056, 1.2045985, 0.4078851, 0.2981652, -0.26469636, 0.4460249, 0.16928293, -0.15559517, -0.7641287, -0.8956279, -0.6098771, -0.87228906, 0.158441, 0.7461226, 0.43037805, -0.7037308, 0.7927367, -1.111509, 0.83980113, 0.6249728, 0.7301589, 0.5632024, 1.7966471, 1.5083653, -1.0206859, -0.11091206, 0.37982145, 1.2463697]",[0.0001988411],0
10250,2023-08-15 21:33:52.851,"[-0.2260837, 0.12802614, -0.8732004, -2.089788, 1.8722432, 2.05272, 0.09746246, 0.49692878, -1.0799059, 0.80176145, -0.26333216, -1.0224636, -0.056668393, -0.060779527, -0.20089072, 1.2798951, -0.55411917, -1.270442, 0.41811302, 0.2239133, 0.3109173, 1.0051724, 0.07736663, 1.7022146, -0.93862903, -0.99493766, -0.68271357, -0.71875495, -1.4715525]",[0.00037947297],0
10251,2023-08-15 21:33:52.851,"[-0.24798988, 0.40499672, 0.49408177, -0.37252557, -0.412961, -0.38151076, -0.042232547, 0.293104, -2.1088455, 0.49807099, 0.8488427, -0.9078823, -0.89734304, 0.8601335, 0.6696, 0.48890257, 1.0623674, -0.5090369, 3.616667, 0.29580164, 0.43410888, 0.8741068, -0.6503351, 0.034938015, 0.96057385, 0.43238926, -0.1975937, -0.04551184, -0.12277038]",[0.00150159],0
10252,2023-08-15 21:33:52.851,"[-2.1694233, -3.1647356, 1.2038506, -0.2649221, 0.0899006, -0.18928011, -0.3495527, 0.17693162, 0.70965016, 0.19469678, -1.4288249, -1.5722991, -1.8213876, -1.2968907, -0.95062584, 0.8917047, 0.751387, -1.5475872, 0.3787144, 0.7525444, -0.03103788, 0.5858543, 2.6022773, -0.5311059, 1.8174038, -0.19327773, 0.94089776, 0.825025, 1.6242892]",[0.00024175644],0


And now we can deploy our pipeline and assign resources to it.  This typically takes about 45 seconds once the command is issued.

In [21]:
from wallaroo.engine_config import Architecture
deployment_config_arm = wallaroo.deployment_config.DeploymentConfigBuilder().cpus(1).memory('1Gi').arch(Architecture.ARM).build()
deployment_config_arm

{'engine': {'cpu': 1,
  'resources': {'limits': {'cpu': 1, 'memory': '1Gi'},
   'requests': {'cpu': 1, 'memory': '1Gi'}},
  'arch': 'arm'},
 'enginelb': {},
 'engineAux': {'images': {}},
 'node_selector': {}}

In [22]:
pipeline.undeploy()
pipeline.deploy(deployment_config=deployment_config_arm)

Waiting for undeployment - this will take up to 45s ..................................... ok
Waiting for deployment - this will take up to 45s ............. ok


name,arm-classification-example
created,2023-08-15 21:27:31.128328+00:00
last_updated,2023-08-15 21:36:57.975903+00:00
deployed,True
tags,
versions,"f0d39e54-1a3a-4d98-8350-938ecd315119, 3f2b6bf2-14eb-4005-80a9-6b0f06ded120, a8d55b45-2331-4d44-aeba-051eaf20e98c"
steps,ccfraudmodel


In [23]:
pipeline.status()

{'status': 'Running',
 'details': [],
 'engines': [{'ip': '10.244.0.22',
   'name': 'engine-84dd7556d7-fphr4',
   'status': 'Running',
   'reason': None,
   'details': [],
   'pipeline_statuses': {'pipelines': [{'id': 'arm-classification-example',
      'status': 'Running'}]},
   'model_statuses': {'models': [{'name': 'ccfraudmodel',
      'version': 'dcbc2816-d54e-4822-9253-acc97e4bf483',
      'sha': 'bc85ce596945f876256f41515c7501c399fd97ebcb9ab3dd41bf03f8937b4507',
      'status': 'Running'}]}}],
 'engine_lbs': [{'ip': '10.244.0.21',
   'name': 'engine-lb-584f54c899-rnmdn',
   'status': 'Running',
   'reason': None,
   'details': []}],
 'sidekicks': []}

We can see our new pipeline with the `status()` command.

In [24]:
start_time = time.time()
result = pipeline.infer_from_file('./cc_data_10k.arrow')
end_time = time.time()
arm_time = end_time - start_time

outputs =  result.to_pandas()
display(outputs)

,time,in.tensor,out.dense_1,check_failures
0,2023-08-15 21:37:12.081,"[-1.0603298, 2.3544967, -3.5638788, 5.138735, -1.2308457, -0.76878244, -3.5881228, 1.8880838, -3.2789674, -3.9563255, 4.099344, -5.653918, -0.8775733, -9.131571, -0.6093538, -3.7480276, -5.0309124, -0.8748149, 1.9870535, 0.7005486, 0.9204423, -0.10414918, 0.32295644, -0.74181414, 0.038412016, 1.0993439, 1.2603409, -0.14662448, -1.4463212]",[0.99300325],0
1,2023-08-15 21:37:12.081,"[-1.0603298, 2.3544967, -3.5638788, 5.138735, -1.2308457, -0.76878244, -3.5881228, 1.8880838, -3.2789674, -3.9563255, 4.099344, -5.653918, -0.8775733, -9.131571, -0.6093538, -3.7480276, -5.0309124, -0.8748149, 1.9870535, 0.7005486, 0.9204423, -0.10414918, 0.32295644, -0.74181414, 0.038412016, 1.0993439, 1.2603409, -0.14662448, -1.4463212]",[0.99300325],0
2,2023-08-15 21:37:12.081,"[-1.0603298, 2.3544967, -3.5638788, 5.138735, -1.2308457, -0.76878244, -3.5881228, 1.8880838, -3.2789674, -3.9563255, 4.099344, -5.653918, -0.8775733, -9.131571, -0.6093538, -3.7480276, -5.0309124, -0.8748149, 1.9870535, 0.7005486, 0.9204423, -0.10414918, 0.32295644, -0.74181414, 0.038412016, 1.0993439, 1.2603409, -0.14662448, -1.4463212]",[0.99300325],0
3,2023-08-15 21:37:12.081,"[-1.0603298, 2.3544967, -3.5638788, 5.138735, -1.2308457, -0.76878244, -3.5881228, 1.8880838, -3.2789674, -3.9563255, 4.099344, -5.653918, -0.8775733, -9.131571, -0.6093538, -3.7480276, -5.0309124, -0.8748149, 1.9870535, 0.7005486, 0.9204423, -0.10414918, 0.32295644, -0.74181414, 0.038412016, 1.0993439, 1.2603409, -0.14662448, -1.4463212]",[0.99300325],0
4,2023-08-15 21:37:12.081,"[0.5817662, 0.09788155, 0.15468194, 0.4754102, -0.19788623, -0.45043448, 0.016654044, -0.025607055, 0.09205616, -0.27839172, 0.059329946, -0.019658541, -0.42250833, -0.12175389, 1.5473095, 0.23916228, 0.3553975, -0.76851654, -0.7000849, -0.11900433, -0.3450517, -1.1065114, 0.25234112, 0.020944182, 0.21992674, 0.25406894, -0.04502251, 0.10867739, 0.25471792]",[0.0010916889],0
...,...,...,...,...
10249,2023-08-15 21:37:12.081,"[-0.90164274, -0.50116056, 1.2045985, 0.4078851, 0.2981652, -0.26469636, 0.4460249, 0.16928293, -0.15559517, -0.7641287, -0.8956279, -0.6098771, -0.87228906, 0.158441, 0.7461226, 0.43037805, -0.7037308, 0.7927367, -1.111509, 0.83980113, 0.6249728, 0.7301589, 0.5632024, 1.7966471, 1.5083653, -1.0206859, -0.11091206, 0.37982145, 1.2463697]",[0.0001988411],0
10250,2023-08-15 21:37:12.081,"[-0.2260837, 0.12802614, -0.8732004, -2.089788, 1.8722432, 2.05272, 0.09746246, 0.49692878, -1.0799059, 0.80176145, -0.26333216, -1.0224636, -0.056668393, -0.060779527, -0.20089072, 1.2798951, -0.55411917, -1.270442, 0.41811302, 0.2239133, 0.3109173, 1.0051724, 0.07736663, 1.7022146, -0.93862903, -0.99493766, -0.68271357, -0.71875495, -1.4715525]",[0.00037947297],0
10251,2023-08-15 21:37:12.081,"[-0.24798988, 0.40499672, 0.49408177, -0.37252557, -0.412961, -0.38151076, -0.042232547, 0.293104, -2.1088455, 0.49807099, 0.8488427, -0.9078823, -0.89734304, 0.8601335, 0.6696, 0.48890257, 1.0623674, -0.5090369, 3.616667, 0.29580164, 0.43410888, 0.8741068, -0.6503351, 0.034938015, 0.96057385, 0.43238926, -0.1975937, -0.04551184, -0.12277038]",[0.00150159],0
10252,2023-08-15 21:37:12.081,"[-2.1694233, -3.1647356, 1.2038506, -0.2649221, 0.0899006, -0.18928011, -0.3495527, 0.17693162, 0.70965016, 0.19469678, -1.4288249, -1.5722991, -1.8213876, -1.2968907, -0.95062584, 0.8917047, 0.751387, -1.5475872, 0.3787144, 0.7525444, -0.03103788, 0.5858543, 2.6022773, -0.5311059, 1.8174038, -0.19327773, 0.94089776, 0.825025, 1.6242892]",[0.00024175644],0


In [25]:
display(f"Standard architecture: {x64_time}")
display(f"ARM architecture: {arm_time}")

'Standard architecture: 0.09396791458129883'

'ARM architecture: 0.07423877716064453'

With our work in the pipeline done, we'll undeploy it to get back our resources from the Kubernetes cluster.  If we keep the same settings we can redeploy the pipeline with the same configuration in the future.

In [26]:
ccfraud_pipeline.undeploy()

Waiting for undeployment - this will take up to 45s ..................................... ok


name,vpzfccfraudpipeline
created,2023-08-09 19:33:35.076256+00:00
last_updated,2023-08-09 19:35:08.065724+00:00
deployed,False
tags,
versions,"76bd9d2c-677c-4a2e-ad9c-994545c87a40, 020505b3-f203-41f5-9c71-3e227ec20ac1"
steps,vpzfccfraudmodel


And there we have it!  Feel free to use this as a template for other models, inferences and pipelines that you want to deploy with Wallaroo!